# A system of advection-diffusion-reaction equations

$$
\frac{\partial u_1}{\partial t} + w \cdot \nabla u_1 - \nabla \cdot (\epsilon \nabla u_1) = f_1 - K u_1 u_2
$$
$$
\frac{\partial u_2}{\partial t} + w \cdot \nabla u_2 - \nabla \cdot (\epsilon \nabla u_2) = f_2 - K u_1 u_2
$$
$$
\frac{\partial u_3}{\partial t} + w \cdot \nabla u_3 - \nabla \cdot (\epsilon \nabla u_3) = f_3 + K u_1 u_2 - K u_3
$$

This system models the chemical reaction between two species A and B in some domain $\Omega$
$$
A + B \rightarrow C
$$
We assume that the reaction is *first-order*, meaning that the reaction rate is proportional to the concentrations $[A]$ and $[B]$ of the two species:
$$
\frac{d}{dt} [C] = K[A][B]
$$

$$
u_1 = [A], u_2 = [B], u_3 = [C]
$$

The chemical reactions take part at each point in the domain $\Omega$. In addition, we assume that the species $A, B, C$ diffuse throughout the domain with **diffusivity** $\epsilon$ and are advected with velocity $w$.

To make things visually and physically interesting, we shall let the chemical reaction take place in the velocity field computed from the solution of imcompressible NS Eq. around a cylinder. i.e.)
$$
\rho \left(\frac{\partial w}{\partial t} + w \cdot \nabla w \right) = \nabla \cdot \sigma(w, p) + f
$$
$$
\nabla \cdot w = 0
$$
$$
\frac{\partial u_1}{\partial t} + w \cdot \nabla u_1 - \nabla \cdot (\epsilon \nabla u_1) = f_1 - K u_1 u_2
$$
$$
\frac{\partial u_2}{\partial t} + w \cdot \nabla u_2 - \nabla \cdot (\epsilon \nabla u_2) = f_2 - K u_1 u_2
$$
$$
\frac{\partial u_3}{\partial t} + w \cdot \nabla u_3 - \nabla \cdot (\epsilon \nabla u_3) = f_3 + K u_1 u_2 - K u_3
$$

We assume that $u_1 = u_2 = u_3 = 0$ at $t = 0$ and inject the species $A, B$ into the system by specifying non-zero source terms $f_1, f_2$ close to the corners at the inflow, and $f_3 = 0$. The result will be that $A, B$ are convected by advection and diffusion throughout the channel, and when they mix the species $C$ will be formed.

Since the system is one-way coupled from the NS subsystem to the advection-diffusion-reaction subsystem, we do not need to recompute the solution to NS Eq., but can just **read back** the previously computed velocity field and **feed into** out equations. (Learn how to read and write solutions)

## Variational Formulation
We obtain the variational formulation of our system by multiplying each equation by a test function, integrating the second-order terms $-\nabla \cdot (\epsilon \nabla u_i)$ by parts, and summing up the equations.

It is convenient to think of the PDE system as a vector of equations. The test functions are collected in a vector too, and the variational formulation is the inner product of the vector PDE and the vector test function.

Discretization in time: backward Euler method => time derivative $\frac{u_i^{n + 1} - u_i^{n}}{\Delta t}$.

Let $v_1, v_2, v_3$ be teh test functions.

The inner product results in:
$$
\int_\Omega (\frac{u_1^{n + 1} - u_1^{n}}{\Delta t} v_1 + w \cdot \nabla u_1^{n+1} v_1 + \epsilon \nabla u_1^{n+1} \cdot \nabla v_1) dx +
\int_\Omega (\frac{u_2^{n + 1} - u_2^{n}}{\Delta t} v_2 + w \cdot \nabla u_2^{n+1} v_2 + \epsilon \nabla u_2^{n+1} \cdot \nabla v_2) dx + 
\int_\Omega (\frac{u_3^{n + 1} - u_3^{n}}{\Delta t} v_3 + w \cdot \nabla u_3^{n+1} v_3 + \epsilon \nabla u_3^{n+1} \cdot \nabla v_3) dx - \int_\Omega (f_1 v_1 + f_2 v_2 + f_3 v_3) dx - \int_\Omega (-K u_1^{n+1} u_2^{n+1}v_1 -K u_1^{n+1} u_2^{n+1} v_2 + K u_1^{n+1} u_2^{n+1} v_3 -K u_3^{n+1} v_3) dx = 0
$$

For this problem it is natural to assume homogeneous Neumann Boundary Conditions on the entire boundary $\frac{\partial u_i}{\partial n} = 0$. The boundary terms vanish when we integrate by parts.

In [1]:
import gmsh
import os
import numpy as np
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
from mpi4py import MPI
from petsc4py import PETSc
from dolfinx import fem, plot, graph, geometry, io, log
from dolfinx.fem import petsc as fem_petsc
from dolfinx.nls import petsc as nls_petsc
import dolfinx.cpp.mesh as cpp_mesh
import dolfinx.mesh as dolfin_mesh
import ufl
import adios4dolfinx
import pyvista
from pyvista.plotting.utilities import active_scalars_algorithm

In [2]:
# Specify the domain (mesh)
gdim = 2
gmsh_model_rank = 0
mesh_comm = MPI.COMM_WORLD
msh, cell_markers, facet_markers = io.gmshio.read_from_msh("mesh_flow.msh", mesh_comm, gmsh_model_rank, gdim=gdim)
facet_markers.name = "Facet markers"

Info    : Reading 'mesh_flow.msh'...
Info    : 11 entities
Info    : 7584 nodes
Info    : 2064 elements
Info    : Done reading 'mesh_flow.msh'


In [5]:
t = 0
T = 8                     # Final time
dt = 1 / 1600                 # Time step size
num_steps = int(T / dt)
Delta_t = fem.Constant(msh, PETSc.ScalarType(dt))
eps = fem.Constant(msh, PETSc.ScalarType(0.01))  # diffusivity
K = fem.Constant(msh, PETSc.ScalarType(10.0))  # rate of reaction

# Choose type of function space
W = fem.functionspace(
    mesh=msh, 
    element=ufl.VectorElement( # For Vector Function Space using Vector Element
        family="Lagrange", 
        cell=msh.ufl_cell(), 
        degree=2
    )
)

P = ufl.FiniteElement(
    family="Lagrange", 
    cell=msh.ufl_cell(), 
    degree=1
)
V = fem.functionspace(
    mesh=msh, 
    element=ufl.MixedElement([P, P, P])
)

# Variational Formulation
# Note that as the problem is non-linear, we have replace the TrialFunction with a Function, 
# which serves as the unknown of our problem
w = fem.Function(W)
adios4dolfinx.read_function(w, "./Flows/flow_function_0")
u, (v1, v2, v3) = fem.Function(V), ufl.TestFunctions(V)
u.x.array[:] = 0.0
u1, u2, u3 = ufl.split(u)

u_n = fem.Function(V)
u_n.x.array[:] = 0.0
u_n1, u_n2, u_n3 = ufl.split(u_n)

f = fem.Function(V)
f1, f2, f3 = f.sub(0).collapse(), f.sub(1).collapse(), f.sub(2).collapse()

x = ufl.SpatialCoordinate(msh)
def source(x, target, r):
    result = np.zeros_like(x[0])
    values = (x[0] - target[0])**2 + (x[1] - target[1])**2
    true_mask = values < (r * r)
    false_mask = np.logical_not(true_mask)
    result[true_mask] = 0.1
    result[false_mask] = 0.0
    return result
f1.interpolate(lambda x : source(x, np.array([0.1, 0.1]), 0.05))
f2.interpolate(lambda x : source(x, np.array([0.1, 0.3]), 0.05))
f3.x.array[:] = 0.0

F = ((u1 - u_n1) / Delta_t * v1 + ufl.dot(w, ufl.grad(u1)) * v1 + eps * ufl.dot(ufl.grad(u1), ufl.grad(v1))) * ufl.dx
F += ((u2 - u_n2) / Delta_t * v2 + ufl.dot(w, ufl.grad(u2)) * v2 + eps * ufl.dot(ufl.grad(u2), ufl.grad(v2))) * ufl.dx
F += ((u3 - u_n3) / Delta_t * v3 + ufl.dot(w, ufl.grad(u3)) * v3 + eps * ufl.dot(ufl.grad(u3), ufl.grad(v3))) * ufl.dx
F -= (f1 * v1 + f2 * v2 + f3 * v3) * ufl.dx
F -= (-K * u1 * u2 * v1 - K * u1 * u2 * v2 + K * u1 * u2 * v3 - K * u3 * v3) * ufl.dx

problem = fem_petsc.NonlinearProblem(F, u)

# Get a solver
solver = nls_petsc.NewtonSolver(MPI.COMM_WORLD, problem)
solver.convergence_criterion = "incremental"
solver.rtol = 1e-6
solver.report = True

ksp = solver.krylov_solver
opts = PETSc.Options()
option_prefix = ksp.getOptionsPrefix()
opts[f"{option_prefix}ksp_type"] = "preonly"
opts[f"{option_prefix}pc_type"] = "lu"
opts[f"{option_prefix}pc_factor_mat_solver_type"] = "mumps"
ksp.setFromOptions()

# log.set_log_level(log.LogLevel.INFO)

# Visualization for gif
pyvista.start_xvfb()
V1, V1_to_V = V.sub(0).collapse()
V2, V2_to_V = V.sub(1).collapse()
V3, V3_to_V = V.sub(2).collapse()

grid1 = pyvista.UnstructuredGrid(*plot.vtk_mesh(V1))
grid2 = pyvista.UnstructuredGrid(*plot.vtk_mesh(V2))
grid3 = pyvista.UnstructuredGrid(*plot.vtk_mesh(V3))

grid1["u1"] = u.sub(0).collapse().x.array
grid2["u2"] = u.sub(1).collapse().x.array
grid3["u3"] = u.sub(2).collapse().x.array

plotter = pyvista.Plotter(shape=(3, 1), notebook=True)
plotter.open_movie("u_time_chemistry.mp4")
plotter.subplot(0, 0)
plotter.add_text(f"time: {t:.2f}", font_size=12, name="timelabel")
plotter.add_text("Chemical A", font_size=9, position="left_edge")
magma = pyvista.LookupTable(cmap="magma")
magma.scalar_range = (0, 0.03)
magma.above_range_color = 'r'
plotter.add_mesh(active_scalars_algorithm(grid1, "u1"), show_edges=True, show_scalar_bar=True, cmap=magma)
plotter.subplot(1, 0)
plotter.add_text("Chemical B", font_size=9, position="left_edge")
cool = pyvista.LookupTable(cmap="cool")
cool.scalar_range = (0, 0.03)
cool.above_range_color = 'b'
plotter.add_mesh(active_scalars_algorithm(grid2, "u2"), show_edges=True, show_scalar_bar=True, cmap=cool)
plotter.subplot(2, 0)
plotter.add_text("Chemical C", font_size=9, position="left_edge")
lut = pyvista.LookupTable()
lut.scalar_range = (0, 1.8e-4)
lut.above_range_color = 'g'
plotter.add_mesh(active_scalars_algorithm(grid3, "u3"), show_edges=True, show_scalar_bar=True, cmap=lut)
plotter.link_views()
plotter.view_xy()
plotter.camera.zoom(3)

progress = tqdm(desc="Solving Chemistry PDE", total=num_steps)
for i in range(num_steps):
    progress.update(1)
    adios4dolfinx.read_function(w, f"./Flows/flow_function_{i}")

    try:
        n, converged = solver.solve(u)
    except Exception as error:
        print("An error occurred:", type(error).__name__) # An error occurred: NameError
        continue
    # print(f"Step {i+1}: num iterations: {n}")

    # Update variable with solution form this time step
    u_n.x.array[:] = u.x.array
    u.x.scatter_forward()
    # For animation
    grid1["u1"] = u.sub(0).collapse().x.array
    grid2["u2"] = u.sub(1).collapse().x.array
    grid3["u3"] = u.sub(2).collapse().x.array
    plotter.subplot(0, 0)
    plotter.add_text(f"time: {t:.2f}", font_size=12, name="timelabel")
    plotter.render()
    plotter.subplot(1, 0)
    plotter.render()
    plotter.subplot(2, 0)
    plotter.render()
    plotter.write_frame()
    t += dt
    
plotter.close()

Solving Chemistry PDE:   0%|          | 0/12800 [00:00<?, ?it/s]